# Week 3: Forecaster & Reporter Testing

This notebook tests the full workflow for:
- Forecaster Agent (personalized scenarios)
- Reporter Agent (HTML email generation)

**Prerequisites:**
- XAI_API_KEY in config/.env (optional, fallback works)
- SMTP config in config/.env (optional for email test)

In [ ]:
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / "src"))

from dotenv import load_dotenv
load_dotenv(Path.cwd().parent / "config" / ".env")

print("✅ Environment loaded")

## 1. Test Forecaster Agent

In [ ]:
from agents.forecaster import ForecasterAgent
from core.grok_client import GrokClient

# Initialize Forecaster
try:
    grok = GrokClient()
    print("✅ Grok client initialized")
except Exception as e:
    print(f"⚠️ Grok unavailable: {e}")
    grok = None

forecaster = ForecasterAgent(grok_client=grok)
print(f"✅ Forecaster initialized (Grok available: {forecaster.grok is not None})")

In [ ]:
# Test basic forecast generation
result = forecaster.execute({
    "current_age": 21,
    "current_value": 5000,
    "monthly_contribution": 300,
    "annual_bonus": 1000,
    "target_ages": [31, 41, 51]
})

print(f"\n{'='*60}")
print(f"Success: {result['success']}")
print(f"Grok Available: {result['grok_available']}")
print(f"Forecasts Generated: {len(result['forecasts'])}")
print(f"{'='*60}\n")

# Display summary
print("📊 SUMMARY:")
print(result['summary'])
print()

In [ ]:
# Display detailed forecasts
import pandas as pd

print("\n📈 DETAILED FORECASTS:\n")

for forecast in result['forecasts']:
    print(f"\n🎯 AGE {forecast['target_age']} ({forecast['years_ahead']} years ahead)")
    print(f"   Base Case:       €{forecast['base_case']:,.0f}")
    print(f"   Bull Case:       €{forecast['bull_case']:,.0f}")
    print(f"   Super-Bull Case: €{forecast['super_bull_case']:,.0f}")
    
    if forecast.get('base_rationale'):
        print(f"\n   Base Rationale: {forecast['base_rationale']}")
    if forecast.get('bull_rationale'):
        print(f"   Bull Rationale: {forecast['bull_rationale']}")
    if forecast.get('super_bull_rationale'):
        print(f"   Super-Bull Rationale: {forecast['super_bull_rationale']}")
    
    if forecast.get('key_assumptions'):
        print(f"\n   Key Assumptions:")
        for assumption in forecast['key_assumptions']:
            print(f"   - {assumption}")
    
    print(f"\n   Source: {'Grok 4' if forecast.get('is_grok') else 'Static Calculation'}")
    print(f"   {'-'*60}")

In [ ]:
# Visualize forecasts
import plotly.graph_objects as go

ages = [21] + [f['target_age'] for f in result['forecasts']]
base_values = [5000] + [f['base_case'] for f in result['forecasts']]
bull_values = [5000] + [f['bull_case'] for f in result['forecasts']]
super_bull_values = [5000] + [f['super_bull_case'] for f in result['forecasts']]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=ages, y=base_values,
    mode='lines+markers',
    name='Base Case',
    line=dict(color='#3498db', width=3)
))

fig.add_trace(go.Scatter(
    x=ages, y=bull_values,
    mode='lines+markers',
    name='Bull Case',
    line=dict(color='#f39c12', width=3)
))

fig.add_trace(go.Scatter(
    x=ages, y=super_bull_values,
    mode='lines+markers',
    name='Super-Bull Case',
    line=dict(color='#2ecc71', width=3)
))

fig.update_layout(
    title="Portfolio Value by Age",
    xaxis_title="Age",
    yaxis_title="Portfolio Value (€)",
    hovermode='x unified',
    height=500
)

fig.show()

## 2. Test Reporter Agent

In [ ]:
from agents.reporter import ReporterAgent
from data.db import Database
from core.portfolio import PortfolioManager

# Initialize Reporter
db = Database()
portfolio = PortfolioManager(db)
reporter = ReporterAgent(db=db, portfolio=portfolio)

print("✅ Reporter initialized")

In [ ]:
# Generate report (without sending email)
report_result = reporter.execute({
    "send_email": False,
    "days_back": 7
})

print(f"\n{'='*60}")
print(f"Success: {report_result['success']}")
print(f"Message: {report_result['message']}")
print(f"HTML Length: {len(report_result['html_report'])} characters")
print(f"{'='*60}\n")

In [ ]:
# Display HTML report (preview)
from IPython.display import HTML

print("📧 HTML EMAIL PREVIEW:\n")
HTML(report_result['html_report'])

In [ ]:
# Save HTML to file for inspection
output_path = Path.cwd().parent / "data" / "test_report.html"
output_path.parent.mkdir(exist_ok=True)

with open(output_path, "w") as f:
    f.write(report_result['html_report'])

print(f"✅ HTML report saved to: {output_path}")
print(f"   Open in browser to view: file://{output_path}")

## 3. Test Email Sending (Optional)

**⚠️ Only run if SMTP is configured in config/.env**

In [ ]:
import os

# Check SMTP configuration
smtp_configured = all([
    os.getenv("SMTP_HOST"),
    os.getenv("SMTP_PORT"),
    os.getenv("SMTP_USER"),
    os.getenv("SMTP_PASSWORD"),
    os.getenv("SMTP_RECIPIENT")
])

if smtp_configured:
    print("✅ SMTP configured")
    print(f"   Host: {os.getenv('SMTP_HOST')}")
    print(f"   Port: {os.getenv('SMTP_PORT')}")
    print(f"   User: {os.getenv('SMTP_USER')}")
    print(f"   Recipient: {os.getenv('SMTP_RECIPIENT')}")
else:
    print("⚠️ SMTP not configured. Skipping email test.")

In [ ]:
# Send test email (uncomment to run)
# if smtp_configured:
#     email_result = reporter.execute({
#         "send_email": True,
#         "days_back": 7
#     })
#     
#     print(f"\n{'='*60}")
#     print(f"Success: {email_result['success']}")
#     print(f"Message: {email_result['message']}")
#     print(f"{'='*60}\n")
# else:
#     print("⚠️ Skipped: SMTP not configured")

## 4. Integration Test: Full Pipeline

In [ ]:
# Test full workflow: Scout → Analyst → Forecaster → Reporter
from agents.scout import ScoutAgent
from agents.analyst import AnalystAgent

print("🔄 Running full pipeline...\n")

# 1. Scout for signals
scout = ScoutAgent()
scout_result = scout.execute({"days_back": 1, "max_results": 3})
print(f"✅ Scout: {len(scout_result.get('articles', []))} signals found")

# 2. Analyze with Analyst
analyst = AnalystAgent(grok_client=grok)
analyst_result = analyst.execute({
    "articles": scout_result.get('articles', []),
    "max_analyses": 2
})
print(f"✅ Analyst: {len(analyst_result.get('analyses', []))} analyses generated")

# 3. Generate forecasts
forecast_result = forecaster.execute({
    "current_age": 21,
    "target_ages": [31]
})
print(f"✅ Forecaster: {len(forecast_result.get('forecasts', []))} forecasts generated")

# 4. Generate report
final_report = reporter.execute({"send_email": False})
print(f"✅ Reporter: HTML report generated ({len(final_report['html_report'])} chars)")

print(f"\n{'='*60}")
print("🎉 FULL PIPELINE COMPLETE!")
print(f"{'='*60}")

## 5. Summary

**Week 3 Components Tested:**
- ✅ Forecaster Agent (Grok + fallback)
- ✅ Reporter Agent (HTML generation)
- ✅ Jinja template rendering
- ✅ Full pipeline integration

**Next Steps:**
- Configure SMTP for email delivery
- Set up cron job for weekly reports
- Build Orchestrator Agent (Week 4)